In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import sys
import os
import glob
from keras.preprocessing import image
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy
from scipy import misc
import scipy.io

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def sort(path, im_type, backend):

    images = glob.glob1(path,"*." + str(im_type))
    im_type_length = len(im_type) + 1
    images = [element[:-im_type_length] for element in images]
    images.sort(key=int)
    images = [element + "." + str(im_type) for element in images]

    if backend == "keras" or backend == "Keras" or backend == "keras":
        images = [np.array(image.load_img(path + "/" + fname)) for fname in images]
    if backend == "opencv" or backend == "openCV" or backend == "OPENCV" or backend == "cv2":
        images = [np.array(cv2.imread(path + "/" + fname)) for fname in images]
    if backend == "PIL" or backend == "pil":
        images = [np.array(Image.open(path + "/" + fname)) for fname in images]
    return images

In [3]:
def save_images(aug_path, aug_imgs):
    for i in range(len(aug_imgs)):
        aug = misc.toimage(aug_imgs[i])
        aug.save(aug_path + "/" + str(i + 1) + ".png")

In [4]:
def sort_list(path_list, im_type, backend):
    final = []
    for path in path_list:
        final.append(sort(path, im_type, backend))
    return final

In [5]:
def multiple_image_numpy(imgs, rows, cols):
    total = len(imgs)
    new = np.ndarray((total, rows, cols, 3), dtype=np.uint8)

    for i in range(total):
        new[i] = imgs[i]
    
    return new


In [6]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [7]:
def use_stats(imgs, masks, jump):
    rl = []
    bl = []
    gl = []
    for i in range(len(imgs)):
        img = imgs[i]
        mask = masks[i]
        r,g,b = get_rgb_medians(img, mask)
        rl.append(r)
        bl.append(b)
        gl.append(g)
    rl = np.array(rl)
    bl = np.array(bl)
    gl = np.array(gl)
    r_l = np.percentile(rl, 50-jump)
    r_m = np.percentile(rl, 50)
    r_h = np.percentile(rl, 50+jump)
    g_l = np.percentile(gl, 50-jump)
    g_m = np.percentile(gl, 50)
    g_h = np.percentile(gl, 50+jump)
    b_l = np.percentile(bl, 50-jump)
    b_m = np.percentile(bl, 50)
    b_h = np.percentile(bl, 50+jump)
    final = [r_l, r_m, r_h, g_l, g_m, g_h, b_l, b_m, b_h]
    return final

In [8]:
def show_stats(imgs, discs):
    rl = []
    bl = []
    gl = []
    for i in range(len(imgs)):
        img = imgs[i]
        disc = discs[i]
        r,g,b = get_rgb_medians(img, disc)
        rl.append(r)
        bl.append(b)
        gl.append(g)
    rl = np.array(rl)
    bl = np.array(bl)
    gl = np.array(gl)
    r_25 = np.percentile(rl, 20)
    r_75 = np.percentile(rl, 80)
    g_25 = np.percentile(gl, 20)
    g_75 = np.percentile(gl, 80)
    b_25 = np.percentile(bl, 20)
    b_75 = np.percentile(bl, 80)
    r_50 = np.percentile(rl, 50)
    g_50 = np.percentile(gl, 50)
    b_50 = np.percentile(bl, 50)
    print("Stats")
    print("Red 25th Percentile " + str(r_25))
    print("Red 50th Percentile " + str(r_50))
    print("Red 75th Percentile " + str(r_75))
    print("Green 25th Percentile " + str(g_25))
    print("Green 50th Percentile " + str(g_50))
    print("Green 75th Percentile " + str(g_75))
    print("Blue 25th Percentile " + str(b_25))
    print("Blue 50th Percentile " + str(b_50))
    print("Blue 75th Percentile " + str(b_75))

In [9]:
def get_rgb_lists(img, mask):
    img = cv2.GaussianBlur(img,(5,5),0)
    if (len(mask.shape) == 3):
        mask = rgb2gray(mask)
    img[mask < 7] = [0, 0, 0]
    image = Image.fromarray(img)
    r = []
    g = []
    b = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            color = image.getpixel((i, j))
            if not color[0] == 0: r.append(color[0])
            if not color[1] == 0: g.append(color[1])
            if not color[2] == 0: b.append(color[2])
    return r,g,b

In [10]:
def get_rgb_medians(img, disc):
    r,g,b = get_rgb_lists(img, disc)
    r,g,b = np.median(r), np.median(g), np.median(b)
    return r,g,b

In [11]:
def get_rgb_percentiles(img, disc, percentile):
    r,g,b = get_rgb_lists(img, disc)
    r,g,b = np.percentile(r, percentile), np.percentile(g, percentile), np.percentile(b, percentile)
    return r,g,b

In [12]:
lens_cups = sort(path = "/Users/karimabedrabbo/Desktop/work2/lens/cropped_cups_png", im_type = "png", backend = "keras")
lens_discs = sort(path = "/Users/karimabedrabbo/Desktop/work2/lens/cropped_discs_png", im_type = "png", backend = "keras")
lens_vessels = sort(path = "/Users/karimabedrabbo/Desktop/work2/lens/cropped_vessels_png", im_type = "png", backend = "keras")
lens_orig = sort(path = "/Users/karimabedrabbo/Desktop/work2/lens/cropped_orig_png", im_type = "png", backend = "keras")

In [13]:
cups_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_not_glauc_cups_png", im_type = "png", backend = "keras")
discs_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_not_glauc_discs_png", im_type = "png", backend = "keras")
vessels_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_not_glauc_vessels_png", im_type = "png", backend = "keras")
orig_not_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_not_glauc_orig_png", im_type = "png", backend = "keras")

In [14]:
cups_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_glauc_cups_png", im_type = "png", backend = "keras")
discs_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_glauc_discs_png", im_type = "png", backend = "keras")
vessels_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_glauc_vessels_png", im_type = "png", backend = "keras")
orig_glauc = sort(path = "/Users/karimabedrabbo/Desktop/work2/cropped_glauc_orig_png", im_type = "png", backend = "keras")

In [15]:
show_stats(lens_orig, lens_cups)
show_stats(lens_orig, lens_vessels)
show_stats(lens_orig, lens_discs)

Stats
Red 25th Percentile 109.0
Red 50th Percentile 131.0
Red 75th Percentile 163.8
Green 25th Percentile 53.6
Green 50th Percentile 87.0
Green 75th Percentile 97.4
Blue 25th Percentile 29.0
Blue 50th Percentile 48.0
Blue 75th Percentile 55.8
Stats
Red 25th Percentile 66.60000000000001
Red 50th Percentile 109.0
Red 75th Percentile 131.2
Green 25th Percentile 24.800000000000004
Green 50th Percentile 49.0
Green 75th Percentile 62.6
Blue 25th Percentile 10.600000000000001
Blue 50th Percentile 30.0
Blue 75th Percentile 38.800000000000004
Stats
Red 25th Percentile 68.6
Red 50th Percentile 103.0
Red 75th Percentile 133.60000000000002
Green 25th Percentile 29.0
Green 50th Percentile 60.0
Green 75th Percentile 72.0
Blue 25th Percentile 14.8
Blue 50th Percentile 38.0
Blue 75th Percentile 42.0


In [79]:
def moder_aug_single(image, image_masks, model_list, model_masks, sigma):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    for i in range(len(image_masks)):
        mask = image_masks[i]
        if (i == 0):
            image[mask < 7] = 0
        else:
            image[mask > 248] = 0
        
    irl, igl, ibl = get_rgb_percentiles(image, image, 48)
    irm, igm, ibm = get_rgb_percentiles(image, image, 50)
    irh, igh, ibh = get_rgb_percentiles(image, image, 52)
    
    image_info = [irl, irm, irh, igl, igm, igh, ibl, ibm, ibh]
    
    model_info = use_stats(model_list, model_masks, 30)
    
    rl, gl, bl = model_info[0] - image_info[0], model_info[3] - image_info[3], model_info[6] - image_info[6]
    rm, gm, bm = model_info[1] - image_info[1], model_info[4] - image_info[4], model_info[7] - image_info[7]
    rh, gh, bh = model_info[2] - image_info[2], model_info[5] - image_info[5], model_info[8] - image_info[8]
    
    if(rl > rh):
        rl,rh = rh,rl
    if(gl > gh):
        gl,gh = gh,gl
    if(bl > bh):
        bl,bh = bh,bl
    if(rh > 0):
        rh = 0
    if(gh > 0):
        gh = 0
    if(bh > 0):
        bh = 0
    
    rl, rm, rh, gl, gm, gh, bl, bm, bh = rl * sigma, rm * sigma, rh * sigma, gl * 1.01 * sigma, gm * 1.01 *sigma, gh * 1.01 * sigma, bl * sigma, bm * sigma, bh * sigma
    
    print(image_info)
    print(model_info)
    print(rl,rm,rh,gl,gm,gh,bl,bm,bh)
    
    seq = iaa.Sequential(
        [
            iaa.WithChannels(channels=[0],children=iaa.Add((rm,rm))),
            iaa.WithChannels(channels=[1],children=iaa.Add((gm,gm))),
            iaa.WithChannels(channels=[2],children=iaa.Add((bm,bm))),
            #iaa.WithChannels(channels=[0],children=iaa.Multiply(((r-sigma)/r, (r-sigma)/r))),
            #iaa.WithChannels(channels=[1],children=iaa.Multiply(((g-(sigma*.573))/g, (g-(sigma*.573))/g))),
            #iaa.WithChannels(channels=[2],children=iaa.Multiply(((b-(sigma*.246) )/b, (b-(sigma*.246))/b))),
            #iaa.ContrastNormalization((1.0004, 1.0004)),
            #iaa.Multiply((0.8, 0.9)),
            #iaa.Fliplr(0.5), # horizontally flip 50% of all images
            #iaa.OneOf([
            #            iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
            #            iaa.AverageBlur(k=(2, 3)), # blur image using local means with kernel sizes between 2 and 7
            #            iaa.MedianBlur(k=(1, 5)), # blur image using local medians with kernel sizes between 2 and 7
            #        ]),
            #sometimes(iaa.Affine(
            #    scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, # scale images to 80-120% of their size, individually per axis
            #    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
            #    rotate=(-15, 15), # rotate by -45 to +45 degrees
            #    order=[0], # use nearest neighbour or bilinear interpolation (fast)
            #    cval=(0, 0), # if mode is constant, use a cval between 0 and 255
            #    mode='constant' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            #))
        ],
        random_order=False
    )
    image = seq.augment_image(image)
    return image

In [87]:
test_vessels = np.copy(vessels_not_glauc[0])
test_cup = np.copy(cups_not_glauc[0])
test_disc = np.copy(discs_not_glauc[0])
test_orig_cup = np.copy(orig_not_glauc[0])
test_orig_disc = np.copy(orig_not_glauc[0])
test_orig_vessels = np.copy(orig_not_glauc[0])
test_orig_last = np.copy(orig_not_glauc[0])

In [88]:
vessels_aug = moder_aug_single(test_orig_vessels, [test_vessels], lens_orig, lens_vessels, 1.0)

[109.0, 111.0, 113.0, 52.0, 53.0, 54.0, 25.0, 25.0, 26.0]
[66.60000000000001, 109.0, 131.2, 24.800000000000004, 49.0, 62.6, 10.600000000000001, 30.0, 38.800000000000004]
(-42.39999999999999, -2.0, 0.0, -27.471999999999994, -4.04, 0.0, -14.399999999999999, 5.0, 0.0)


In [89]:
cup_aug = moder_aug_single(test_orig_cup, [test_cup, test_vessels], lens_orig, lens_cups, 1.0)

[190.0, 191.0, 192.0, 125.0, 126.0, 128.0, 71.0, 72.0, 72.0]
[109.0, 131.0, 163.8, 53.6, 87.0, 97.4, 29.0, 48.0, 55.8]
(-81.0, -60.0, -28.19999999999999, -72.114, -39.39, -30.905999999999995, -42.0, -24.0, -16.200000000000003)


In [90]:
disc_aug = moder_aug_single(test_orig_disc, [test_disc, test_cup, test_vessels], lens_orig, lens_discs, 1.0)

[154.0, 156.0, 157.0, 81.0, 82.0, 82.0, 37.0, 37.0, 38.0]
[68.6, 103.0, 133.60000000000002, 29.0, 60.0, 72.0, 14.8, 38.0, 42.0]
(-85.4, -53.0, -23.399999999999977, -52.52, -22.22, -10.1, -22.2, 1.0, 0.0)


In [91]:
augmentation_parts = [disc_aug, cup_aug, vessels_aug]

In [92]:
for part in augmentation_parts:
    partim = Image.fromarray(part)
    for i in range(part.shape[0]):
        for j in range(part.shape[1]):
            color = partim.getpixel((i, j))
            if not (color[0] < 7 and color[1] < 7 and color[2] < 7):
                print(color)
                test_orig_last[i, j, :] = color
            


(116, 80, 50)
(116, 80, 50)
(114, 78, 48)
(111, 75, 45)
(111, 75, 45)
(113, 77, 47)
(115, 79, 49)
(114, 78, 46)
(115, 79, 47)
(114, 78, 48)
(116, 77, 48)
(116, 77, 48)
(115, 76, 47)
(114, 74, 45)
(110, 70, 41)
(108, 72, 43)
(117, 81, 49)
(117, 81, 49)
(118, 82, 50)
(118, 82, 50)
(120, 82, 50)
(117, 81, 51)
(118, 82, 52)
(116, 80, 50)
(113, 77, 47)
(112, 76, 44)
(114, 78, 46)
(116, 80, 48)
(115, 79, 47)
(115, 79, 47)
(115, 79, 47)
(117, 79, 47)
(117, 79, 47)
(117, 77, 46)
(116, 76, 45)
(112, 70, 42)
(112, 73, 44)
(111, 75, 45)
(111, 75, 45)
(109, 75, 44)
(109, 74, 45)
(108, 73, 46)
(107, 70, 46)
(116, 80, 50)
(120, 81, 52)
(118, 80, 48)
(118, 80, 48)
(119, 81, 49)
(119, 81, 49)
(120, 82, 50)
(120, 82, 50)
(121, 83, 51)
(121, 83, 51)
(119, 83, 53)
(121, 85, 55)
(119, 83, 53)
(116, 80, 50)
(115, 79, 47)
(116, 80, 48)
(118, 82, 50)
(118, 82, 50)
(117, 81, 49)
(117, 81, 49)
(119, 81, 49)
(119, 81, 49)
(120, 80, 49)
(118, 78, 47)
(114, 72, 42)
(115, 75, 44)
(114, 76, 44)
(112, 76, 44)
(112, 

(93, 53, 32)
(92, 52, 33)
(87, 51, 32)
(80, 49, 33)
(73, 46, 32)
(67, 44, 33)
(60, 41, 31)
(108, 67, 42)
(135, 101, 66)
(133, 99, 66)
(132, 96, 64)
(131, 95, 63)
(129, 93, 61)
(128, 89, 60)
(127, 88, 59)
(127, 87, 58)
(125, 85, 56)
(125, 82, 56)
(123, 80, 54)
(121, 78, 52)
(118, 75, 49)
(115, 72, 46)
(117, 72, 46)
(115, 70, 44)
(113, 68, 42)
(113, 66, 43)
(112, 65, 42)
(110, 62, 42)
(109, 61, 41)
(107, 59, 39)
(106, 58, 40)
(104, 58, 39)
(102, 56, 37)
(101, 54, 38)
(98, 54, 37)
(98, 54, 37)
(98, 54, 37)
(95, 53, 35)
(93, 51, 31)
(90, 50, 31)
(86, 50, 31)
(81, 48, 33)
(74, 47, 33)
(67, 44, 31)
(59, 40, 30)
(108, 69, 43)
(108, 67, 42)
(133, 99, 66)
(131, 97, 64)
(132, 96, 64)
(131, 95, 63)
(130, 91, 62)
(128, 89, 60)
(127, 87, 58)
(125, 85, 56)
(124, 83, 56)
(123, 80, 54)
(122, 79, 53)
(118, 75, 49)
(114, 71, 45)
(116, 71, 45)
(116, 69, 44)
(115, 68, 43)
(114, 67, 42)
(112, 65, 42)
(111, 64, 41)
(109, 61, 41)
(108, 60, 40)
(107, 59, 41)
(105, 57, 39)
(103, 57, 38)
(100, 53, 37)
(97, 53, 

(125, 64, 50)
(127, 64, 51)
(127, 64, 49)
(128, 63, 46)
(126, 61, 44)
(126, 61, 42)
(126, 64, 46)
(125, 63, 45)
(122, 60, 42)
(122, 59, 44)
(121, 58, 43)
(119, 57, 39)
(120, 55, 38)
(121, 56, 39)
(120, 55, 38)
(120, 53, 37)
(119, 52, 36)
(48, 38, 29)
(48, 40, 30)
(50, 42, 32)
(49, 41, 31)
(83, 60, 37)
(90, 63, 38)
(97, 66, 40)
(102, 67, 40)
(104, 68, 39)
(107, 68, 40)
(105, 66, 40)
(105, 66, 40)
(104, 63, 38)
(104, 61, 37)
(106, 60, 37)
(108, 61, 38)
(108, 61, 38)
(108, 61, 38)
(111, 61, 39)
(111, 61, 37)
(111, 61, 37)
(113, 62, 37)
(114, 61, 37)
(116, 61, 35)
(118, 61, 36)
(120, 63, 38)
(114, 64, 42)
(117, 64, 43)
(120, 62, 43)
(124, 58, 39)
(124, 51, 37)
(132, 49, 36)
(132, 51, 37)
(131, 52, 38)
(131, 52, 40)
(130, 53, 40)
(130, 53, 40)
(130, 57, 41)
(129, 58, 41)
(127, 61, 42)
(126, 61, 42)
(121, 61, 40)
(120, 60, 39)
(119, 57, 37)
(119, 52, 36)
(125, 59, 47)
(127, 60, 46)
(123, 56, 40)
(122, 54, 38)
(122, 56, 37)
(123, 56, 40)
(122, 55, 39)
(119, 54, 37)
(120, 55, 40)
(121, 56, 41)

(66, 47, 37)
(64, 44, 36)
(59, 41, 32)
(55, 37, 30)
(52, 36, 29)
(53, 37, 30)
(53, 37, 30)
(50, 37, 31)
(49, 36, 30)
(48, 35, 29)
(49, 36, 30)
(50, 40, 33)
(51, 41, 34)
(52, 42, 35)
(51, 41, 32)
(51, 41, 32)
(53, 41, 32)
(52, 40, 31)
(49, 37, 28)
(78, 66, 47)
(83, 69, 50)
(83, 67, 46)
(82, 65, 42)
(87, 67, 45)
(82, 66, 45)
(80, 62, 42)
(80, 60, 40)
(81, 57, 39)
(82, 54, 37)
(85, 52, 37)
(84, 49, 37)
(84, 45, 34)
(105, 62, 36)
(106, 63, 37)
(106, 63, 37)
(106, 63, 37)
(106, 63, 37)
(105, 62, 36)
(106, 63, 37)
(108, 63, 37)
(108, 63, 37)
(107, 62, 36)
(108, 63, 37)
(108, 61, 36)
(108, 61, 36)
(109, 64, 38)
(108, 63, 37)
(108, 63, 37)
(108, 63, 37)
(108, 63, 37)
(106, 63, 37)
(106, 63, 37)
(104, 63, 38)
(102, 62, 39)
(97, 60, 38)
(92, 58, 39)
(89, 57, 39)
(82, 54, 37)
(78, 51, 36)
(74, 49, 35)
(72, 47, 35)
(71, 46, 32)
(70, 47, 32)
(71, 48, 35)
(71, 48, 35)
(68, 47, 33)
(66, 44, 33)
(62, 43, 31)
(61, 42, 32)
(59, 41, 32)
(57, 42, 32)
(56, 40, 33)
(52, 38, 30)
(49, 35, 29)
(48, 34, 28)
(47

(126, 97, 51)
(127, 98, 52)
(128, 99, 51)
(128, 101, 52)
(133, 106, 57)
(134, 107, 58)
(137, 110, 61)
(137, 113, 61)
(136, 112, 60)
(136, 112, 60)
(137, 113, 61)
(139, 115, 63)
(141, 117, 64)
(142, 118, 65)
(142, 118, 65)
(143, 119, 66)
(144, 120, 68)
(146, 119, 68)
(145, 118, 67)
(143, 119, 66)
(142, 118, 65)
(144, 118, 65)
(144, 118, 65)
(145, 117, 65)
(145, 117, 66)
(144, 114, 64)
(149, 115, 66)
(151, 118, 69)
(150, 120, 70)
(149, 121, 70)
(148, 121, 70)
(144, 120, 68)
(146, 124, 71)
(147, 123, 71)
(147, 123, 71)
(146, 122, 72)
(146, 122, 72)
(147, 123, 73)
(146, 122, 72)
(144, 119, 72)
(141, 116, 69)
(139, 114, 67)
(139, 111, 67)
(140, 111, 67)
(140, 109, 66)
(141, 106, 64)
(139, 101, 60)
(136, 98, 57)
(140, 91, 57)
(134, 87, 53)
(131, 84, 50)
(129, 84, 47)
(126, 84, 46)
(125, 83, 45)
(128, 83, 46)
(130, 84, 47)
(132, 83, 47)
(131, 81, 44)
(133, 84, 47)
(132, 86, 48)
(133, 87, 49)
(131, 87, 48)
(130, 86, 47)
(129, 85, 46)
(127, 83, 44)
(107, 58, 24)
(107, 60, 26)
(108, 66, 28)
(114

...

(153, 61, 36)
(153, 62, 37)
(148, 51, 32)
(146, 49, 30)
(144, 51, 31)
(144, 51, 31)
(142, 51, 30)
(139, 50, 31)
(136, 51, 30)
(134, 53, 32)
(131, 56, 33)
(128, 60, 37)
(123, 61, 36)
(119, 63, 38)
(114, 64, 37)
(140, 59, 37)
(140, 52, 34)
(138, 46, 29)
(136, 42, 26)
(137, 39, 24)
(137, 39, 24)
(137, 39, 24)
(138, 40, 25)
(139, 41, 26)
(139, 43, 27)
(141, 43, 28)
(144, 46, 31)
(169, 73, 46)
(167, 68, 42)
(168, 66, 41)
(169, 64, 40)
(170, 65, 39)
(171, 66, 40)
(173, 68, 42)
(177, 73, 44)
(198, 103, 69)
(196, 103, 68)
(194, 99, 65)
(197, 98, 65)
(196, 98, 67)
(192, 101, 66)
(193, 113, 74)
(183, 68, 45)
(180, 65, 42)
(177, 62, 41)
(174, 59, 38)
(176, 60, 42)
(175, 59, 41)
(171, 55, 38)
(171, 53, 37)
(172, 54, 38)
(166, 53, 35)
(164, 50, 35)
(162, 50, 34)
(160, 50, 33)
(158, 47, 33)
(156, 48, 33)
(157, 49, 34)
(155, 50, 28)
(154, 52, 29)
(157, 54, 33)
(159, 58, 36)
(158, 59, 36)
(155, 60, 36)
(153, 61, 36)
(148, 53, 31)
(147, 50, 31)
(148, 48, 32)
(143, 50, 30)
(142, 51, 30)
(141, 50, 29)
(1

(105, 60, 35)
(98, 41, 29)
(98, 41, 29)
(97, 40, 28)
(97, 38, 27)
(96, 37, 26)
(98, 40, 26)
(100, 42, 28)
(103, 43, 28)
(106, 46, 31)
(112, 50, 33)
(116, 55, 34)
(121, 60, 37)
(172, 71, 39)
(174, 68, 38)
(175, 65, 38)
(178, 63, 40)
(177, 60, 38)
(176, 59, 37)
(175, 60, 37)
(175, 63, 37)
(176, 66, 39)
(177, 69, 39)
(167, 77, 48)
(166, 72, 44)
(164, 68, 43)
(163, 64, 40)
(162, 61, 39)
(161, 58, 37)
(159, 58, 38)
(157, 57, 37)
(154, 57, 36)
(154, 59, 37)
(153, 60, 40)
(153, 60, 40)
(153, 60, 38)
(152, 59, 37)
(150, 57, 35)
(151, 54, 35)
(151, 54, 35)
(150, 53, 34)
(149, 52, 31)
(149, 52, 31)
(148, 51, 30)
(147, 50, 29)
(147, 50, 29)
(147, 52, 30)
(147, 54, 32)
(146, 58, 34)
(146, 61, 38)
(144, 63, 39)
(140, 65, 41)
(135, 65, 39)
(132, 64, 38)
(122, 60, 41)
(118, 59, 39)
(114, 57, 36)
(108, 55, 33)
(104, 56, 32)
(104, 57, 33)
(104, 59, 36)
(94, 40, 27)
(94, 37, 26)
(94, 37, 26)
(96, 39, 28)
(96, 39, 28)
(97, 40, 29)
(98, 42, 27)
(100, 44, 29)
(103, 45, 29)
(106, 48, 30)
(110, 53, 32)
(113,

(179, 89, 52)
(178, 88, 51)
(180, 87, 51)
(180, 87, 49)
(181, 87, 49)
(184, 87, 50)
(182, 85, 48)
(181, 82, 46)
(178, 79, 43)
(177, 78, 42)
(179, 74, 42)
(177, 72, 40)
(176, 71, 39)
(175, 70, 38)
(174, 70, 39)
(175, 71, 40)
(175, 74, 42)
(175, 75, 45)
(174, 76, 45)
(175, 79, 50)
(175, 81, 51)
(175, 81, 51)
(175, 79, 52)
(174, 78, 51)
(174, 75, 49)
(172, 73, 47)
(172, 70, 45)
(171, 69, 44)
(170, 68, 41)
(168, 66, 39)
(167, 65, 38)
(167, 65, 38)
(167, 65, 38)
(169, 68, 38)
(169, 69, 39)
(170, 70, 40)
(169, 71, 40)
(170, 72, 41)
(169, 74, 42)
(168, 74, 42)
(167, 73, 41)
(167, 73, 41)
(163, 73, 42)
(163, 73, 44)
(163, 73, 44)
(162, 72, 43)
(161, 71, 42)
(161, 71, 42)
(160, 70, 41)
(160, 70, 41)
(159, 72, 42)
(158, 71, 41)
(157, 70, 40)
(155, 69, 41)
(153, 67, 40)
(149, 63, 38)
(144, 57, 35)
(139, 54, 31)
(134, 51, 33)
(130, 47, 29)
(127, 45, 27)
(126, 44, 28)
(123, 44, 27)
(124, 46, 29)
(119, 53, 35)
(114, 51, 32)
(110, 48, 31)
(106, 47, 29)
(106, 47, 29)
(106, 47, 29)
(104, 48, 29)
(95, 3

(157, 77, 40)
(156, 73, 39)
(152, 61, 38)
(148, 57, 34)
(147, 56, 31)
(148, 57, 32)
(153, 63, 36)
(157, 67, 40)
(160, 71, 35)
(165, 76, 40)
(167, 78, 42)
(164, 81, 45)
(108, 51, 29)
(107, 48, 30)
(105, 43, 28)
(102, 38, 26)
(100, 33, 24)
(101, 32, 25)
(100, 31, 24)
(100, 32, 25)
(99, 34, 25)
(100, 37, 25)
(100, 40, 27)
(103, 58, 35)
(102, 57, 34)
(101, 55, 35)
(102, 55, 35)
(99, 52, 32)
(99, 50, 33)
(98, 49, 32)
(97, 48, 31)
(97, 48, 29)
(97, 48, 29)
(96, 49, 29)
(108, 66, 40)
(104, 61, 38)
(102, 59, 38)
(102, 59, 40)
(96, 59, 30)
(102, 60, 33)
(109, 61, 34)
(116, 62, 34)
(122, 60, 35)
(128, 57, 33)
(132, 55, 33)
(135, 54, 32)
(138, 53, 32)
(140, 55, 32)
(143, 58, 35)
(147, 62, 37)
(150, 66, 37)
(151, 70, 38)
(154, 73, 39)
(156, 76, 39)
(155, 76, 39)
(153, 74, 37)
(155, 76, 39)
(157, 77, 40)
(155, 74, 40)
(152, 64, 38)
(149, 61, 37)
(148, 57, 34)
(147, 56, 33)
(149, 58, 33)
(153, 62, 37)
(157, 67, 40)
(160, 72, 34)
(163, 77, 39)
(165, 79, 41)
(107, 49, 31)
(105, 45, 30)
(103, 39, 27)
(

(101, 48, 28)
(103, 52, 31)
(102, 53, 32)
(102, 53, 32)
(100, 53, 33)
(97, 51, 33)
(97, 51, 33)
(98, 52, 26)
(98, 51, 27)
(98, 44, 29)
(95, 41, 26)
(96, 40, 25)
(96, 40, 25)
(97, 41, 24)
(97, 44, 26)
(98, 47, 26)
(99, 48, 27)
(97, 48, 27)
(97, 48, 29)
(96, 49, 29)
(95, 48, 28)
(97, 48, 29)
(98, 49, 30)
(101, 49, 31)
(100, 48, 30)
(101, 48, 30)
(102, 49, 29)
(102, 51, 30)
(102, 51, 30)
(100, 51, 30)
(100, 51, 30)
(101, 52, 31)
(101, 52, 31)
(103, 52, 31)
(102, 51, 30)
(102, 51, 30)
(98, 49, 28)
(97, 48, 27)
(97, 48, 29)
(97, 50, 30)
(97, 50, 30)
(97, 50, 30)
(95, 49, 31)
(88, 38, 27)
(84, 34, 23)
(85, 35, 24)
(87, 37, 26)
(87, 37, 26)
(86, 36, 25)
(87, 37, 26)
(89, 39, 28)
(100, 44, 27)
(101, 45, 28)
(100, 47, 27)
(101, 48, 28)
(97, 43, 30)
(95, 41, 28)
(94, 40, 25)
(96, 42, 27)
(97, 44, 26)
(97, 44, 26)
(97, 45, 27)
(95, 46, 27)
(96, 47, 28)
(96, 49, 29)
(96, 49, 29)
(94, 47, 27)
(95, 48, 28)
(97, 48, 29)
(99, 47, 29)
(99, 47, 29)
(98, 46, 28)
(99, 47, 29)
(101, 49, 31)
(97, 48, 29)
(9

In [93]:
Image.fromarray(test_orig_last).show()